In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from globalModule.globalModule import *

In [38]:
__name__ = "RNN"

# Model

In [39]:
def getModel():
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
    model.add(LSTM(128, return_sequences=True, activation='relu'))
    model.add(LSTM(64, return_sequences=False, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(actions.shape[0], activation='softmax'))

    model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['categorical_accuracy'])


    return model

In [40]:
def getData():
    labelMap = {label:num for num, label in enumerate(actions)}
    sequences, labels = [], []
    for action in actions:
        for sequence in np.array(os.listdir(os.path.join(gesturesDataPath, action))).astype(int):
            window = []
            for frame_num in range(sequenceLen):
                try:
                    res = np.load(os.path.join(gesturesDataPath, action, str(sequence), "{}.npy".format(frame_num)))
                    window.append(res)
                except Exception as e:
                    print(e)
            sequences.append(window)
            labels.append(labelMap[action])
    
    return sequences, labels 

In [41]:
sequences, labels = getData()
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [42]:
### INITS

tb_callback = TensorBoard(log_dir=log_dir)

In [43]:
model = getModel()
model.fit(X_train, y_train, epochs=400, callbacks=[tb_callback])

Epoch 1/400
12/12 [==============================] - 3s 20ms/step - loss: 0.6762 - categorical_accuracy: 0.2105
Epoch 2/400
12/12 [==============================] - 0s 21ms/step - loss: 0.6150 - categorical_accuracy: 0.4763
Epoch 3/400
12/12 [==============================] - 0s 20ms/step - loss: 0.5539 - categorical_accuracy: 0.5132
Epoch 4/400
12/12 [==============================] - 0s 21ms/step - loss: 0.4353 - categorical_accuracy: 0.5263
Epoch 5/400
12/12 [==============================] - 0s 27ms/step - loss: 0.3433 - categorical_accuracy: 0.5316
Epoch 6/400
12/12 [==============================] - 0s 23ms/step - loss: 0.3369 - categorical_accuracy: 0.5474
Epoch 7/400
12/12 [==============================] - 0s 26ms/step - loss: 0.2973 - categorical_accuracy: 0.5316
Epoch 8/400
12/12 [==============================] - 0s 22ms/step - loss: 0.2562 - categorical_accuracy: 0.5368
Epoch 9/400
12/12 [==============================] - 0s 31ms/step - loss: 0.2389 - categorical_accuracy:

KeyboardInterrupt: 

In [ ]:

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            48896     
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 5)                 8

In [ ]:
model.save(f"{modelPath}/{__name__}/test.h5")